In [ ]:
#!pip3 install matplotlib

In [2]:
import pandas as pd
import numpy as np
# import csv
import matplotlib as mpt
import os

In [8]:
# data_path = input("include relative path to the dataset: (should work for windows/mac: )")
data_path = "Data Sets/Calgary_Transit_Ridership_20240118-2.csv"
read_file = pd.read_csv(data_path)
read_file.describe

<bound method NDFrame.describe of      Year Month Adult Book of Tickets Ridership  \
0    2023   Dec                             NaN   
1    2023   Nov                         258,261   
2    2023   Oct                         261,861   
3    2023   Sep                         298,214   
4    2023   Aug                         268,624   
..    ...   ...                             ...   
163  2010   May                             NaN   
164  2010   Apr                             NaN   
165  2010   Mar                             NaN   
166  2010   Feb                             NaN   
167  2010   Jan                             NaN   

    Adult Cash-Single Ticket Ride Ridership Adult Day Pass Ridership  \
0                                       NaN                      NaN   
1                                   885,165                   33,071   
2                                   936,216                   42,668   
3                                   976,732                   56,